In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import warnings
from collections import Counter
from sklearn.feature_selection import mutual_info_classif
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTEN
from collections import Counter

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, classification_report

In [13]:
print('\nLoading files ...')
df=pd.read_csv('Clean_loan.csv')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head(3)


Loading files ...


,Primary_applicant_age_in_years,Number_of_dependents,Years_at_current_residence,Foreign_worker,Months_loan_taken_for,Principal_loan_amount,EMI_rate_in_percentage_of_disposable_income,Has_coapplicant,Has_guarantor,Number_of_existing_loans_at_this_bank,...,Marital_status_single,Housing_own,Housing_rent,Employment_status_skilled employee / official,Employment_status_unemployed / unskilled - non-resident,Employment_status_unskilled - resident,Loan_history_critical/pending loans at other banks,Loan_history_delay in paying off loans in the past,Loan_history_existing loans paid back duly till now,Loan_history_no loans taken/all loans paid back duly
0,67.0,1.0,4.0,1.0,6.0,1169000.0,4.0,0.0,0.0,2.0,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,22.0,1.0,2.0,1.0,48.0,5951000.0,2.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,49.0,2.0,3.0,1.0,12.0,2096000.0,2.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


# Dimentionality Reduction Using PCA:

In [14]:
x=df.drop('high_risk_applicant', axis=1)
y=df['high_risk_applicant']

### Handling unbalanced Data:

<div class="alert alert-block alert-info">
<b>Why?:</b> When the data is heavily unbalanced, it trains one segment of the data heavily and the accuracy also increases but it is not a good measure for the performance at that time.
</div>

<div class="alert alert-block alert-success">
<b>Up to you:</b> For further info, Click 
</div>
this is [Text](https://stats.stackexchange.com/a/283942)

In [17]:
sampler = SMOTEN(random_state=0)
X_res, y_res = sampler.fit_resample(x, y)
print(f"Class counts after resampling {Counter(y_res)}")

Class counts after resampling Counter({0.0: 700, 1.0: 700})


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_res,
                                                    y_res,
                                                    test_size=0.2,
                                                    random_state=0)

X_train.shape, X_test.shape

((1120, 27), (280, 27))

In [19]:
target_names = np.unique(y_train)
print('\nThere are %d unique target valuess in this dataset:' % (len(target_names)), target_names)


There are 2 unique target valuess in this dataset: [0. 1.]


In [20]:
scaler = StandardScaler()

scaler.fit(X_train)

StandardScaler()

In [21]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled

array([[-0.81902649, -0.35284273, -0.72513557, ..., -0.26016054,
         0.88833405, -0.22065615],
       [-0.5503359 , -0.35284273, -0.72513557, ..., -0.26016054,
         0.88833405, -0.22065615],
       [-0.90859002, -0.35284273, -0.72513557, ..., -0.26016054,
         0.88833405, -0.22065615],
       ...,
       [-0.99815355, -0.35284273,  0.18128389, ..., -0.26016054,
         0.88833405, -0.22065615],
       [-0.19208178, -0.35284273, -0.72513557, ..., -0.26016054,
        -1.12570266, -0.22065615],
       [-0.19208178,  2.83412386,  0.18128389, ...,  3.84378042,
        -1.12570266, -0.22065615]])

In [22]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)
X_train_scaled.describe()

,Primary_applicant_age_in_years,Number_of_dependents,Years_at_current_residence,Foreign_worker,Months_loan_taken_for,Principal_loan_amount,EMI_rate_in_percentage_of_disposable_income,Has_coapplicant,Has_guarantor,Number_of_existing_loans_at_this_bank,...,Marital_status_single,Housing_own,Housing_rent,Employment_status_skilled employee / official,Employment_status_unemployed / unskilled - non-resident,Employment_status_unskilled - resident,Loan_history_critical/pending loans at other banks,Loan_history_delay in paying off loans in the past,Loan_history_existing loans paid back duly till now,Loan_history_no loans taken/all loans paid back duly
count,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,...,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03,1.120000e+03
mean,1.100806e-16,3.053113e-17,7.523744e-17,-1.992454e-17,7.315577e-17,-1.625684e-17,1.645509e-17,2.146596e-16,-1.754053e-16,1.415534e-16,...,-3.578487e-16,6.998370e-17,2.583251e-16,1.570173e-16,2.481646e-16,1.950820e-16,-2.264062e-16,-3.326704e-16,-6.423433e-17,2.535174e-16
std,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,...,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00,1.000447e+00
min,-1.266844e+00,-3.528427e-01,-1.631555e+00,-6.244998e+00,-1.418957e+00,-9.452689e-01,-1.911607e+00,-1.874037e-01,-1.973855e-01,-6.253814e-01,...,-1.038230e+00,-1.498792e+00,-4.705040e-01,-1.236211e+00,-1.278043e-01,-4.215539e-01,-5.842237e-01,-2.601605e-01,-1.125703e+00,-2.206562e-01
25%,-7.294630e-01,-3.528427e-01,-7.251356e-01,1.601282e-01,-7.547359e-01,-6.072303e-01,-9.931172e-01,-1.874037e-01,-1.973855e-01,-6.253814e-01,...,-1.038230e+00,-1.498792e+00,-4.705040e-01,-1.236211e+00,-1.278043e-01,-4.215539e-01,-5.842237e-01,-2.601605e-01,-1.125703e+00,-2.206562e-01
50%,-2.816453e-01,-3.528427e-01,1.812839e-01,1.601282e-01,-2.565702e-01,-3.372953e-01,8.438626e-01,-1.874037e-01,-1.973855e-01,-6.253814e-01,...,9.631775e-01,6.672041e-01,-4.705040e-01,8.089236e-01,-1.278043e-01,-4.215539e-01,-5.842237e-01,-2.601605e-01,8.883340e-01,-2.206562e-01
75%,5.244265e-01,-3.528427e-01,1.087703e+00,1.601282e-01,2.415955e-01,2.345279e-01,8.438626e-01,-1.874037e-01,-1.973855e-01,1.213010e+00,...,9.631775e-01,6.672041e-01,-4.705040e-01,8.089236e-01,-1.278043e-01,-4.215539e-01,1.711673e+00,-2.601605e-01,8.883340e-01,-2.206562e-01
max,3.748714e+00,2.834124e+00,1.087703e+00,1.601282e-01,4.226921e+00,5.042899e+00,8.438626e-01,5.336074e+00,5.066228e+00,4.889793e+00,...,9.631775e-01,6.672041e-01,2.125380e+00,8.089236e-01,7.824463e+00,2.372176e+00,1.711673e+00,3.843780e+00,8.883340e-01,4.531938e+00


In [23]:
n_comp = 20
# PCA
print('\nRunning PCA ...')
pca = PCA(n_components=n_comp, svd_solver='full', random_state=1001)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
print('Explained variance: %.4f' % pca.explained_variance_ratio_.sum())

print('Individual variance contributions:')
for j in range(n_comp):
    print(pca.explained_variance_ratio_[j])


Running PCA ...
Explained variance: 0.9347
Individual variance contributions:
0.12842943105901408
0.08066639773762145
0.07397864599507345
0.06801471106184362
0.0578306591872688
0.051602366489378694
0.04856015448575603
0.04393299123331342
0.04126455769694672
0.0402416482438396
0.03832626769730664
0.03665224691598452
0.03394232039753584
0.03352104191554683
0.03172929258936923
0.02944881513240232
0.028449495533944144
0.026474539813100596
0.021089391767314225
0.020590438021357623


<div class="alert alert-block alert-success">
<b>Success:</b> As we can see that with 20 features, we can explain almost 94% of the variance. so we will keep 20 features out of 27 independent features
</div>

/////As we can see that with 20 features, we can explain almost 94% of the variance. so we will keep 20 features out of 27 independent features/////

In [24]:
X_train_pca.shape

(1120, 20)

# Model training:

## 1. Random Forest:
<div class="alert alert-block alert-info">
<b>**Reason:**</b> We are not starting with linear models because there are a bit of correlations(which is less so the columns are not dropped)
The main reason I think Random Forest will work better because the data has less randomness (that is the data columns are in a range/fixed values) so tree based will work better.

We will not use linear models as the data is not linear as we saw in the visualization part.

**Note**: We did not apply log-transformation or other transformations because we are not using Linear models and tree based models are not affected by the distribution of the data.
</div>

In [39]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
n_estimators = [20,60,100,120]
max_features = [0.2,0.6,1.0]
max_depth = [2,8,None]
max_samples = [0.5,0.75,1.0]

param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
              'max_samples':max_samples
             }
print(param_grid)

{'n_estimators': [20, 60, 100, 120], 'max_features': [0.2, 0.6, 1.0], 'max_depth': [2, 8, None], 'max_samples': [0.5, 0.75, 1.0]}


In [44]:
rf = RandomForestClassifier()

In [45]:
from sklearn.model_selection import GridSearchCV

rf_grid = GridSearchCV(estimator = rf, 
                       param_grid = param_grid, 
                       cv = 5, 
                       verbose=2, 
                       n_jobs = -1)

In [46]:
rf_grid.fit(X_train_pca,y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [2, 8, None],
                         'max_features': [0.2, 0.6, 1.0],
                         'max_samples': [0.5, 0.75, 1.0],
                         'n_estimators': [20, 60, 100, 120]},
             verbose=2)

In [47]:
rf_grid.best_params_

{'max_depth': None,
 'max_features': 0.2,
 'max_samples': 0.75,
 'n_estimators': 100}

In [48]:
rf_grid.best_score_

0.7803571428571429

In [ ]:
y_pred = rf_grid.best_estimator_.predict(X_test_pca)

In [50]:
cm=confusion_matrix(y_test,y_pred)
print(cm)
print(classification_report(y_test,y_pred))

[[103  38]
 [ 39 100]]
              precision    recall  f1-score   support

         0.0       0.73      0.73      0.73       141
         1.0       0.72      0.72      0.72       139

    accuracy                           0.73       280
   macro avg       0.72      0.72      0.72       280
weighted avg       0.72      0.72      0.72       280



<div class="alert alert-block alert-warning">
<b>Note:</b> The Recall is balacned and good considering the size of the data.
</div>

Lets try one ensambling technique.
# AdaBoost:

In [51]:
from sklearn.ensemble import AdaBoostClassifier

In [52]:
grid = dict()
grid['n_estimators'] = [10, 50, 100, 500]
grid['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]
grid['algorithm'] = ['SAMME', 'SAMME.R']
grid_search = GridSearchCV(estimator=AdaBoostClassifier(), param_grid=grid, n_jobs=-1, cv=10, scoring='accuracy')
grid_result = grid_search.fit(X_train_pca,y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.758036 using {'algorithm': 'SAMME', 'learning_rate': 1.0, 'n_estimators': 500}


In [54]:
y_pred = grid_search.best_estimator_.predict(X_test_pca)
cm=confusion_matrix(y_test,y_pred)
print(cm)
print(classification_report(y_test,y_pred))

[[ 88  53]
 [ 29 110]]
              precision    recall  f1-score   support

         0.0       0.75      0.62      0.68       141
         1.0       0.67      0.79      0.73       139

    accuracy                           0.71       280
   macro avg       0.71      0.71      0.71       280
weighted avg       0.71      0.71      0.71       280



<div class="alert alert-block alert-info">
<b>Note:</b> In this case, the False-Negative is less which is good. But the other values are bad compared to random forest.
</div>

Okay so now that this is not working that well, lets do some model building by our own self in another note book. we will name the folder as models and also create our own performance measures.